#Setup

Importação de bibliotecas, pacotes, módulos e carregamento de arquivos necessários.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.exceptions import ConvergenceWarning
import warnings

warnings.filterwarnings("ignore", category=ConvergenceWarning)


# Importação da base da academia

In [2]:
input_file = "database.csv"
df = pd.read_csv(input_file)
df.head()


,aluno_id,Nome,Objetivo,Idade,TempoComoAluno,CheckinsSemana_1,CheckinsSemana_2,CheckinsSemana_3,CheckinsSemana_4,CheckinsSemana_5,...,AulasGrupo_M2,AulasGrupo_M3,DiasAtrasoPagamento_M1,DiasAtrasoPagamento_M2,DiasAtrasoPagamento_M3,VariacaoPeso_2m,VariacaoGordura_2m,VariacaoCarga_2m,Status,unidade
0,1,Ana Silva,hipertrofia,28,10,3,4,4,5,4,...,1,2,0,1,-1,1.2,0.2,4.4,Ativo,centro
1,2,Bruno Souza,emagrecimento,35,6,4,4,3,3,3,...,2,3,3,5,6,-3.2,-2.1,1.0,Cancelou,zona sul
2,3,Carla Gomes,saude,41,8,2,2,3,3,3,...,1,1,0,0,1,-0.4,-0.2,2.1,Ativo,zona leste
3,4,Diego Santos,condicionamento,27,12,3,3,3,4,4,...,2,1,1,3,2,0.3,-0.1,3.4,Ativo,zona oeste
4,5,Eduarda Lima,bem-estar,32,5,1,2,2,2,2,...,2,1,2,4,5,0.1,-0.4,1.0,Cancelou,zona norte


#Pré-processamento

In [3]:
# Retira colunas não utilizadas no treino
df_model = df.drop(columns=["Nome", "aluno_id", "unidade"])

# Converte Status para binário
df_model["Status"] = df_model["Status"].map({"Cancelou": 1, "Ativo": 0})

# Remove a coluna de objetivo (vai entrar depois como variável categórica one-hot)
df_model = df_model.drop(columns=["Objetivo"])

#df_model.head()


##Normalização

In [4]:
num_cols = df_model.drop(columns=["Status"]).columns

scaler = StandardScaler()
df_model[num_cols] = scaler.fit_transform(df_model[num_cols])

real_data = df_model.copy()

#real_data.head()


##CTGAN

In [5]:
!pip install ctgan --quiet

from ctgan import CTGAN

# Somente Status é categórico
discrete_columns = ['Status']

ctgan = CTGAN(
    epochs=300,
    batch_size=16,
    generator_dim=(128,128),
    discriminator_dim=(128,128),
    pac=1,
    verbose=True
)

ctgan.fit(real_data, discrete_columns)

synthetic_data = ctgan.sample(10000)

#synthetic_data.head()


  0%|                                                                                                                                 | 0/300 [00:00<?, ?it/s]

Gen. (0.00) | Discrim. (0.00):   0%|                                                                                                  | 0/300 [00:00<?, ?it/s]

Gen. (0.42) | Discrim. (-0.00):   0%|                                                                                                 | 0/300 [00:00<?, ?it/s]

Gen. (0.42) | Discrim. (-0.00):   0%|▎                                                                                        | 1/300 [00:00<00:31,  9.43it/s]

Gen. (0.69) | Discrim. (0.01):   0%|▎                                                                                         | 1/300 [00:00<00:31,  9.43it/s]

...

Gen. (-0.36) | Discrim. (-0.55):  99%|█████████████████████████████████████████████████████████████████████████████████████▏| 297/300 [00:26<00:00, 10.30it/s]

Gen. (-0.27) | Discrim. (-1.07):  99%|█████████████████████████████████████████████████████████████████████████████████████▏| 297/300 [00:27<00:00, 10.30it/s]

Gen. (-0.27) | Discrim. (-1.07): 100%|█████████████████████████████████████████████████████████████████████████████████████▋| 299/300 [00:27<00:00, 10.19it/s]

Gen. (-0.45) | Discrim. (-0.47): 100%|█████████████████████████████████████████████████████████████████████████████████████▋| 299/300 [00:27<00:00, 10.19it/s]

Gen. (-0.45) | Discrim. (-0.47): 100%|██████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:27<00:00, 11.08it/s]

##Separação treino/teste

In [6]:
#dados sintéticos usados para treino e teste
X_synth = synthetic_data.drop(columns=["Status"])
y_synth = synthetic_data["Status"]

X_train, X_test, y_train, y_test = train_test_split(
    X_synth, y_synth, test_size=0.2, random_state=42
)

#dados reais usados apenas para avaliação (validação cruzada) e predição final (csv)
X_real = real_data.drop(columns=["Status"])
y_real = real_data["Status"]

# Árvore de Decisão e Regressão Logística

In [7]:
models = {
    "Árvore de Decisão": DecisionTreeClassifier(max_depth=5, random_state=42),
    "Regressão Logística": LogisticRegression(
        penalty="l1", solver="saga", max_iter=10000, random_state=42
    )
}

for name, model in models.items():
    #treina com os dados sintéticos
    model.fit(X_train, y_train)

    #predição no teste sintético
    pred = model.predict(X_test)
    pred_proba = model.predict_proba(X_test)[:, 1]

    print("\n====================")
    print(f"MODELO: {name}")
    print("====================")
    print("Acurácia :", accuracy_score(y_test, pred))
    print("Precisão :", precision_score(y_test, pred))
    print("Recall   :", recall_score(y_test, pred))
    print("F1-score :", f1_score(y_test, pred))



MODELO: Árvore de Decisão
Acurácia : 0.981
Precisão : 0.9788359788359788
Recall   : 0.9809119830328739
F1-score : 0.9798728813559322

MODELO: Regressão Logística
Acurácia : 0.9935
Precisão : 0.9925847457627118
Recall   : 0.9936373276776246
F1-score : 0.9931107578166402


# Validação Cruzada (K-Fold e Leave-One-Out)

In [8]:
for name, model in models.items():
    print("\n" + "#" * 25, name.upper(), "#" * 25)

    print("\n--- K-FOLD (dados reais) ---")
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    print("Acurácia:", np.mean(cross_val_score(model, X_real, y_real, cv=kf, scoring='accuracy')))
    print("Precisão:", np.mean(cross_val_score(model, X_real, y_real, cv=kf, scoring='precision')))
    print("Recall  :", np.mean(cross_val_score(model, X_real, y_real, cv=kf, scoring='recall')))
    print("F1      :", np.mean(cross_val_score(model, X_real, y_real, cv=kf, scoring='f1')))

    print("\n--- LOO (dados reais) ---")
    loo = LeaveOneOut()
    preds, trues = [], []
    for tr, te in loo.split(X_real):
        model.fit(X_real.iloc[tr], y_real.iloc[tr])
        preds.append(model.predict(X_real.iloc[te])[0])
        trues.append(y_real.iloc[te])

    print("Acurácia:", accuracy_score(trues, preds))
    print("Precisão:", precision_score(trues, preds))
    print("Recall  :", recall_score(trues, preds))
    print("F1      :", f1_score(trues, preds))



######################### ÁRVORE DE DECISÃO #########################

--- K-FOLD (dados reais) ---
Acurácia: 0.9833333333333334
Precisão: 0.9


/home/juliaduboisas/Documents/faculdade/empreendAcademiaBack/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Recall  : 0.9
F1      : 0.9

--- LOO (dados reais) ---


Acurácia: 0.9833333333333333
Precisão: 1.0
Recall  : 0.9565217391304348
F1      : 0.9777777777777777

######################### REGRESSÃO LOGÍSTICA #########################

--- K-FOLD (dados reais) ---


Acurácia: 1.0


Precisão: 1.0


Recall  : 1.0


F1      : 1.0

--- LOO (dados reais) ---


Acurácia: 1.0
Precisão: 1.0
Recall  : 1.0
F1      : 1.0


# Previsão final para os alunos REAIS

In [9]:
#escolher o modelo (colocar o melhor modelo/o preferido)
melhor_modelo = models["Regressão Logística"]

#previsões para todos os alunos reais
prob_real = melhor_modelo.predict_proba(X_real)[:, 1]
pred_real = melhor_modelo.predict(X_real)

#montar tabela final com dados reais originais
df_saida = df[["aluno_id", "Nome"]].copy()
df_saida["probabilidade_evasao"] = prob_real
df_saida["predicao"] = pred_real

#ordena do maior risco pro menor
df_saida = df_saida.sort_values(by="probabilidade_evasao", ascending=False)

#exporta o arquivo para o backend
df_saida.to_csv("predictions/predicoes_alunos_academia.csv", index=False)

df_saida.head()


,aluno_id,Nome,probabilidade_evasao,predicao
10,11,Kamila Santos,0.995702,1
30,31,Ana Ribeiro,0.994505,1
35,36,Felipe Azevedo,0.994105,1
50,51,Ursula Melo,0.993474,1
23,24,Yara Ribeiro,0.992022,1
